In [4]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as plt
import control as ct

print('Versão do pandas -> %s' % pd.__version__)
print('Versão do numpy -> %s' % np.__version__)
print('Versão do scipy -> %s' % sp.__version__)
# print('Versão do matplotlib -> %s' % matplotlib.__version__)
print('Versão do control -> %s' % ct.__version__)

Versão do pandas -> 1.5.0
Versão do numpy -> 1.23.4
Versão do scipy -> 1.9.2
Versão do control -> 0.9.3.post2


Em alguns casos, é usado:

```python
from control.matlab import *
```

# <span style=color:green> Construção das representações LTI possíveis

## Espaço de estados

In [6]:
A = [[1, 2], [2, 1]]
B = [[1, 2], [2, 1]]
C = [[1, 2], [2, 1]]
D = [[1, 2], [2, 1]]

sys = ct.ss(A, B, C, D,)
sys

<LinearIOSystem:sys[3]:['u[0]', 'u[1]']->['y[0]', 'y[1]']>